In [3]:
import os

In [4]:
file_name = 'abstraction/cep/abstracted-ground_truth-EventLogXESNoSegment.csv'

In [21]:
file_name = 'abstraction/cep/abstracted-infomap-EventLogXESNoSegment.csv'

In [ ]:
file_name = 'abstraction/cep/abstracted-Louvain_18-EventLogXESNoSegment.csv'


In [ ]:
file_name = 'abstraction/cep/abstracted-dpclus_79.txt-EventLogXESNoSegment.csv'


In [ ]:
file_name = 'abstraction/cep/abstracted-Louvain_23-EventLogXESNoSegment.csv'


In [ ]:
file_name = 'abstraction/cep/abstracted-Louvain_23-EventLogXESNoSegment.csv'


In [2]:
file_name = 'abstraction/ground_truth/ground_truth_abstracted.csv'


In [5]:
# Split the filename and extension
base, ext = os.path.splitext(file_name)

# Only add the new extension if there isn't one already
xes_filename = base + ('.' + "xes" if ext == '' else ".xes")

In [6]:
from conversion import read_csv, format_dataframe, convert_to_event_log, write_xes

In [7]:
def create_abstracted_log2(csv_path, output_file="abstraction/abstracted.xes", sort=False, split_hour=None):
    """Convert a CSV file into a dataframe, segments it on a daily basis, sort it chronologically and save it as XES."""
    abstracted_csv = read_csv(csv_path, sep=',')
    case_concept_name = 'case_id'

    if 'case:concept:name' not in abstracted_csv:
        abstracted_csv['case:concept:name'] = case_concept_name

    if sort:
        abstracted_csv = abstracted_csv.sort_values(by=["time:timestamp", "eventId"])

    date_ref = None
    prev_date = None

    for i in abstracted_csv.index:
        #case_id = case_concept_name
        if split_hour and date_ref is None:
            date_ref = str(abstracted_csv.at[i, 'time:timestamp'])[0:10]
            case_id = date_ref

        # update case concept name if splitting is provided
        if split_hour is not None:
            #hour = abstracted_csv.at[i, 'time:timestamp'].hour # str(abstracted_csv.at[i, 'time:timestamp'])[0:10]
            hour = int(abstracted_csv.at[i, 'time:timestamp'][11:13])
            if hour > split_hour:
                current_date = str(abstracted_csv.at[i, 'time:timestamp'])[0:10]
                if current_date != date_ref:
                    if prev_date is None:
                        prev_date = "-"
                    print(
                        f"Split hour {hour} due to passed split at {split_hour}. Current date: {abstracted_csv.at[i, 'time:timestamp']}, reference date {date_ref}, prev_date {prev_date}")
                    date_ref = current_date
                    case_id = date_ref
        prev_date = abstracted_csv.at[i, 'time:timestamp'][0:15]
        abstracted_csv.at[i, 'case:concept:name'] = case_id#str(abstracted_csv.at[i, 'time:timestamp'])[0:10]

    abstracted_csv = format_dataframe(abstracted_csv, case_id='case:concept:name', activity_key='concept:name',
                                      timestamp_key='time:timestamp')


    abstracted = convert_to_event_log(abstracted_csv)
    write_xes(abstracted, output_file)


In [8]:
create_abstracted_log2(file_name, output_file=xes_filename, sort=True, split_hour=19)


Split hour 20 due to passed split at 19. Current date: 2020-03-17 20:50:00+00:00, reference date 2020-03-16, prev_date 2020-03-17 18:5
Split hour 21 due to passed split at 19. Current date: 2020-03-18 21:05:00+00:00, reference date 2020-03-17, prev_date 2020-03-18 19:2
Split hour 23 due to passed split at 19. Current date: 2020-03-19 23:54:00+00:00, reference date 2020-03-18, prev_date 2020-03-19 18:5
Split hour 20 due to passed split at 19. Current date: 2020-03-20 20:50:00+00:00, reference date 2020-03-19, prev_date 2020-03-20 19:5
Split hour 20 due to passed split at 19. Current date: 2020-03-21 20:09:00+00:00, reference date 2020-03-20, prev_date 2020-03-21 19:3
Split hour 20 due to passed split at 19. Current date: 2020-03-22 20:16:00+00:00, reference date 2020-03-21, prev_date 2020-03-22 18:4
Split hour 23 due to passed split at 19. Current date: 2020-03-23 23:42:00+00:00, reference date 2020-03-22, prev_date 2020-03-23 18:5
Split hour 20 due to passed split at 19. Current date: 

exporting log, completed traces ::   0%|          | 0/18 [00:00<?, ?it/s]